<h2>Imports</h2>

-This version incorporates ML pipeline with fitting n different folds by randomly sampling neutral comments instead of undersampling them 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import progressbar
import nltk
import matplotlib.pyplot as plt
import re

from tqdm.auto import tqdm
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ruddit-jigsaw-dataset/LICENSE
/kaggle/input/ruddit-jigsaw-dataset/README.md
/kaggle/input/ruddit-jigsaw-dataset/requirements.txt
/kaggle/input/ruddit-jigsaw-dataset/ruddit-comment-extraction.ipynb
/kaggle/input/ruddit-jigsaw-dataset/Dataset/create_dataset_variants.py
/kaggle/input/ruddit-jigsaw-dataset/Dataset/identityterms_group.txt
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Ruddit.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/ReadMe.md
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Ruddit_individual_annotations.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/node_dictionary.npy
/kaggle/input/ruddit-jigsaw-dataset/Dataset/post_with_issues.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Thread_structure.txt
/kaggle/input/ruddit-jigsaw-dataset/Dataset/load_node_dictionary.py
/kaggle/input/ruddit-jigsaw-dataset/Dataset/sample_input_file.csv
/kaggle/input/ruddit-jigsaw-dataset/Models/BERT.py
/kaggle/input/ruddi

In [2]:
CONFIG = {"seed":42,
         "nfolds":3}

<h2>Upload Datasets</h2>

In [3]:
train_1 = "/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv"
train_2 = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
train_3 = "/kaggle/input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv"
train_4 = "/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv"

In [4]:

TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
SAMPLE_SUBMISSION = "/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv"

In [5]:
df_severity=  pd.read_csv(train_1)
df_classification=  pd.read_csv(train_2)
df_ruddit=  pd.read_csv(train_3)
df_ruddit = df_ruddit[df_ruddit["txt"]!="[deleted]"]
df_ubias=  pd.read_csv(train_4)

In [6]:
# df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)
# df_validation_data = pd.read_csv(VALID_DATA_PATH)
df_sample_submission = pd.read_csv(SAMPLE_SUBMISSION)

## Process severity data

In [7]:

# total_less_toxic = {}


# for comment in df_severity["less_toxic"].unique():
#     total_less_toxic[comment] = len(df_severity.loc[df_severity["less_toxic"]==comment])
    
# total_more_toxic = {}

# for comment in df_severity["more_toxic"].unique():
#     total_more_toxic[comment] = len(df_severity.loc[df_severity["more_toxic"]==comment,])
    

In [8]:
# unique_comments = np.unique(np.concatenate((list(total_less_toxic.keys()), list(total_more_toxic.keys()))))
# comment_scores = {}
# total_evals = {}
# for comment in unique_comments:
#     total_evals[comment] = total_less_toxic.get(comment,0) + total_more_toxic.get(comment,0)
#     if total_evals[comment]>=3:
#             comment_scores[comment] = (-1*total_less_toxic.get(comment,0) + total_more_toxic.get(comment,0)) / total_evals[comment]
 


In [9]:
# pd.Series(list(total_evals.values())).describe() 

In [10]:
# df_sev = pd.DataFrame(comment_scores.items(), columns=["text","score"])

In [11]:
# df_sev["score"].value_counts()

In [12]:
# max_  = df_sev["score"].max()
# min_ = df_sev["score"].min()
# df_sev["score"] = (df_sev["score"] - min_) / (max_ - min_)

## Process ruddit data

In [13]:
df_ruddit

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...,-0.022
3,42g75o,cza2bw8,The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...,-0.083
5,42g75o,cza2jj3,>Men and women are not payed less for the same...,https://www.reddit.com/r/changemyview/comments...,-0.042
...,...,...,...,...,...
5833,cu67co,f0i0mqp,They should only censor things that talk badly...,https://i.redd.it/kfsmqzxae3i31.jpg/f0i0mqp/,0.064
5834,cganu1,f80wlxq,> and one of them is a woman. \n\nOH SHIT we b...,https://www.reddit.com/r/worldpolitics/comment...,0.458
5835,cu67co,f8uksbp,how is this flared as US politics,https://i.redd.it/kfsmqzxae3i31.jpg/f8uksbp/,-0.292
5836,cganu1,fa6nc1r,People in Hong Kong must decide if they are go...,https://www.reddit.com/r/worldpolitics/comment...,0.333


In [14]:
df_ruddit["offensiveness_score"].describe()

count    5722.000000
mean       -0.027165
std         0.334562
min        -0.889000
25%        -0.255000
50%        -0.062000
75%         0.167000
max         0.979000
Name: offensiveness_score, dtype: float64

In [15]:
df_ruddit["offensiveness_score"].value_counts()

-0.021    161
 0.000    152
-0.167    136
-0.125    128
-0.208    123
         ... 
 0.422      1
-0.761      1
-0.875      1
-0.543      1
 0.578      1
Name: offensiveness_score, Length: 287, dtype: int64

In [16]:
df_ruddit = df_ruddit[["txt","offensiveness_score"]].rename(columns = {"txt":"text","offensiveness_score":"score"})

In [17]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+')  # Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml')  # Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(' +', ' ', text) # Remove Extra Spaces
    text = text.strip().lower() # remove spaces at the beginning and at the end of string and make string lower

    return text




def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ')      
        
    data[col] = data[col].str.lower()
#     data[col] = data[col].str.replace("4", "a") 
#     data[col] = data[col].str.replace("2", "l")
#     data[col] = data[col].str.replace("5", "s") 
#     data[col] = data[col].str.replace("1", "i") 
#     data[col] = data[col].str.replace("!", "i") 
#     data[col] = data[col].str.replace("|", "i") 
#     data[col] = data[col].str.replace("0", "o") 
#     data[col] = data[col].str.replace("l3", "b") 
#     data[col] = data[col].str.replace("7", "t") 
#     data[col] = data[col].str.replace("7", "+") 
#     data[col] = data[col].str.replace("8", "ate") 
#     data[col] = data[col].str.replace("3", "e") 
#     data[col] = data[col].str.replace("9", "g")
#     data[col] = data[col].str.replace("6", "g")
#     data[col] = data[col].str.replace("@", "a")
#     data[col] = data[col].str.replace("$", "s")
#     data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", "i will ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
#     data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ")
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ")
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you')
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", ' ')
    data[col] = data[col].str.replace('\s+', ' ')  # will remove more than one whitespace character
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1') # 2 or more characters are replaced by 2 characters
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
   
    
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")


    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    
    # Remove ip address
    data[col] = data[col].str.replace(r'(([0-9]+\.){2,}[0-9]+)','')
    
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r' +', ' ')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()      
    
    return data


In [18]:
tqdm.pandas()
df_ruddit['text'] = df_ruddit['text'].progress_apply(text_cleaning)

  0%|          | 0/5722 [00:00<?, ?it/s]

In [19]:
df_ruddit = clean(df_ruddit,"text")

In [20]:
pd.set_option("display.max_colwidth",None)
df_ruddit.head(10)

,text,score
0,"the difference in average earnings between men and women can be explained by taking into account relevant factors. so it is not a myth, you just feel that you can explain it.",-0.083
1,"the myth is that the ""gap"" is entirely based on the sex of the person.",-0.022
3,"the assertion is that women get paid less for the same jobs, and that they get paid less because they are women.",-0.146
4,"you said in the op that is not what they are measuring. they are measuring aggregate. as entertainers, women make less. is not that what jlaw was complaining about",-0.083
5,"men and women are not payed less for the same job i do not think many people say this is the case though. i think this is a misconception on the other side, not something feminists have said. even if women have a lower ""average"" wage in all jobs than men.. i mean that is even bigger than individual cases. this shows that women are in an inferior position more than if a woman were to get literally 30 cents less. because averages show the bigger picture. edit changed 70 to 30.",-0.042
6,"all the wage gap is, is an average of total men is pay vs total women is pay. if you believe that is all it is, then sure. so you admit it exists when people refer to the pay gap, that is what they are referring to most of the time. i had say most of the time people think they are talking about discrimination, which is only true if they actually say it is about discrimination. this is essentially critics that are assuming they are saying something other than what they are saying.",-0.021
11,"no, the point is they are not talking about discrimination here.",-0.174
13,so women are paid less for the same job,-0.229
16,"but obviously that does not make sense, does it that women just do not want to earn more money. that conflicts with what we know about humankind. people, men and women both, generally do want to make money and generally are not okay with being given less than other people are given. women are a whole are not just less interested in money than men are. something must be going on.",0.000
17,"i think that hollywood and professional sports are a big exception here, because salaries in those two industries have virtually no cap (and often are not tied to the quality of work or quantity of work at all - they are about subjective ""star power"")",0.098


In [21]:
### prepare n folds of ruddit data
fraction = 0.8 
for fold in range(CONFIG["nfolds"]):
    df = df_ruddit.sample(frac = fraction, random_state = CONFIG["seed"] + fold*10)
    mean = df["score"].mean()
    std = df["score"].std()
    df_ruddit["score"] = (df_ruddit["score"] - mean)/std
    df.to_csv(f'/kaggle/working/df_ruddit_fld{fold}.csv', index=False)
    

## Process Classification data

In [22]:
# Create a score that messure how much toxic is a comment
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

# cat_mtpl = {'obscene': 3, 'toxic': 4, 'threat': 4, 
#             'insult': 2, 'severe_toxic': 4, 'identity_hate': 2}

for category in cat_mtpl:
    df_classification[category] = df_classification[category] * cat_mtpl[category]

df_classification['score'] = df_classification.loc[:, 'toxic':'identity_hate'].sum(axis=1)

# min_len = (df_classification['score'] > 0).sum()  # len of toxic comments



df_classification = df_classification[["comment_text","score"]].rename(columns = {"comment_text":"text"})





# df_y0_undersample = df_classification[df_classification['score'] == 0].sample(n=int(min_len * 1.5), random_state=201)  # take non toxic comments
# df_classification_new = pd.concat([df_classification[df_classification['score'] > 0], df_y0_undersample])  # make new df
# df_classification_new

In [23]:
tqdm.pandas()
df_classification['text'] = df_classification['text'].progress_apply(text_cleaning)

  0%|          | 0/159571 [00:00<?, ?it/s]

In [24]:
df_classification = clean(df_classification,"text")

In [25]:
df_classification["score"].value_counts()

0.00    143346
0.32      5666
1.12      3800
0.48      1758
2.62      1738
0.96      1215
4.12       385
0.16       317
0.64       301
1.82       290
1.98       204
0.80       181
2.46       164
1.50        76
2.14        31
5.62        31
3.32        21
2.30        20
3.48        10
3.96        10
1.66         5
3.96         1
4.82         1
Name: score, dtype: int64

In [26]:
### prepare n folds of classification data

length_of_toxic = len(df_classification[df_classification["score"]>0])
fraction = 0.8 
for fold in range(CONFIG["nfolds"]):
    df_toxic = df_classification[df_classification["score"]>0].sample(frac = fraction, random_state = CONFIG["seed"] + fold*10)
    df_neutral = df_classification[df_classification["score"]==0].sample(n = int(fraction*length_of_toxic), random_state = CONFIG["seed"] + fold*10)
    df = pd.concat([df_neutral, df_toxic]).sample(frac=1, random_state = CONFIG["seed"] + fold*10)
    mean = df["score"].mean()
    std = df["score"].std()
    df["score"] = (df["score"] - mean) / (std)
    df.to_csv(f'/kaggle/working/df_classification_fld{fold}.csv', index=False)
    

## Process unintended Bias Data

In [27]:
pd.set_option("display.max_columns",500)
df_ubias = df_ubias[["target","comment_text"]].rename(columns = {"target":"score", "comment_text":"text"})

In [28]:
df_ubias.head()

,score,text
0,0.000000,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"
1,0.000000,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!"
2,0.000000,This is such an urgent design problem; kudos to you for taking it on. Very impressive!
3,0.000000,Is this something I'll be able to install on my site? When will you be releasing it?
4,0.893617,haha you guys are a bunch of losers.


In [29]:
tqdm.pandas()
df_ubias['text'] = df_ubias['text'].progress_apply(text_cleaning)

  0%|          | 0/1804874 [00:00<?, ?it/s]

In [30]:
df_ubias = clean(df_ubias,"text")

In [31]:
df_ubias["score"].value_counts()

0.000000    1264764
0.166667     138501
0.200000     107492
0.300000      59098
0.400000      50013
             ...   
0.026684          1
0.924561          1
0.007458          1
0.145161          1
0.870088          1
Name: score, Length: 2913, dtype: int64

In [32]:
### prepare n folds of ubias data

length_of_toxic = len(df_ubias[df_ubias["score"]>0])
fraction = 0.2
for fold in range(9):
    df_toxic = df_ubias[df_ubias["score"]>0].sample(frac = fraction, random_state = CONFIG["seed"] + fold*10)
    df_neutral = df_ubias[df_ubias["score"]==0].sample(n = int(fraction*length_of_toxic), random_state = CONFIG["seed"] + fold*10)
    df = pd.concat([df_neutral, df_toxic]).sample(frac=1, random_state = CONFIG["seed"] + fold*10)
    mean = df["score"].mean()
    std = df["score"].std()
    df["score"] = (df["score"] - mean) / (std)
    df.to_csv(f'/kaggle/working/df_ubias_fld{fold}.csv', index=False)
    

### Clean Validation/ Severity data & Test Data

In [33]:
df_val = df_severity.copy()
df_val.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=\nWHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back
1,188,"""And yes, people should recognize that but they usually don't. One of the first objections you hear directed at someone who says he favors free markets is invariably """"There's no such thing as a free market."""" Ridiculously trivial. ""","Daphne Guinness \n\nTop of the mornin' my favourite Alien Life Form. I'm sorry, I can't marry you because you are a poor commoner so, therefore, unimportant. Can I have my dresses back. I hope your fat Oirish arse hasn't stretched them."
2,82,"Western Media?\n\nYup, because every crime in the entire world is caused by the west. Racists.","""Atom you don't believe actual photos of masturbation is sexually explicit photos. I am sorry to say they are sexually explicit and so you said yourself that the disclaimer is used for that purpose. Also, Wikipedia itself is targeted for minors. In addition, you say that Wikipedia does not have child pornography photos. I do believe there is a photo of a real prepubescent child nude (not an illustration) that is in bondage with the title """"Virgin Killer"""". Which recently Wikipedia archived so no more comments could be made. 211.220.23.15 \n\n"""
3,347,"And you removed it! You numbskull! I don't care what you say anymore, this is my life! Go ahead with your own life, leave me alone! —","You seem to have sand in your vagina.\n\nMight want to wash that stinking hole out, bitch."
4,539,smelly vagina \n\nBluerasberry why don't you be a model for a cheesy blue vagina syndrome. A lot are experiencing this vaginal disease.,"hey \n\nway to support nazis, you racist"


In [34]:
tqdm.pandas()
df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

In [35]:
df_val = clean(df_val,"less_toxic")
df_val = clean(df_val,"more_toxic")

In [36]:
df_val.head()

,worker,less_toxic,more_toxic
0,313,this article sucks woo woo woo,what !! ! !! ! !! ! ! ! !! !! wher is your sexy pic gone from your main page put it back
1,188,"""and yes, people should recognize that but they usually do not . one of the first objections you hear directed at someone who says he favors free markets is invariably """"there is no such thing as a free market."""" ridiculously trivial. ""","daphne guinness top of the mornin my favourite alien life form. i am sorry, i cannot marry you because you are a poor commoner so, therefore, unimportant. can i have my dresses back. i hope your fat oirish arse has not stretched them."
2,82,"western media yup, because every crime in the entire world is caused by the west. racists.","""atom you do not believe actual photos of masturbation is sexually explicit photos. i am sorry to say they are sexually explicit and so you said your self that the disclaimer is used for that purpose. also, social medium it self is targeted for minors. in addition, you say that social medium does not have child pornography photos. i do believe there is a photo of a real prepubescent child nude (not an illustration) that is in bondage with the title """"virgin killer"""". which recently social medium archived so no more comments could be made. """
3,347,"and you removed it ! you numbskull ! i do not care what you say anymore, this is my life ! go ahead with your own life, leave me alone ! —","you seem to have sand in your vagina. might want to wash that stinking hole out, bitch."
4,539,smelly vagina bluerasberry why do not you be a model for a cheesy blue vagina syndrome. a lot are experiencing this vaginal disease.,"hey way to support nazis, you racist"


In [37]:
df_test.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether there is an """"anti-editorializing"""" policy here. There is, and it's called wikipedia:neutral point of view. It discusses at some length the case of what we should do when writing about a subject which most of us find repugnant. Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.\n\nIf something you write is edited and you're not sure why, please continue to question such edits on the talk page. Sometimes, you'll learn more about wikipedia policy. Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"""
1,732895,"Looks like be have an abuser , can you please look into this? thanks."
2,1139051,"I confess to having complete (and apparently blissful) ignorance of Jordan, but I've glanced at the article. Is this a woman or a soap opera!?. I don't think there was much to change in terms of the description of the various diseases. It is mentioned that she is famous for the size of her breasts: am I correct in assuming this is because they are grotesquely large rather than vanishingly small? 04:09 11 Jul 2003 (UTC)"
3,1434512,"""\n\nFreud's ideas are certainly much discussed today, and I would be the first to agree that they must be grappled with and dealt with seriously, if only because of their currency. So Freud deserves a long and thorough article in Wikipedia. I believe that a balanced article would include A) explanations of Freud's theories, and B) the status of Freudianism today. The existing article is quite thorough on """"A"""" and had almost nothing on """"B"""". A neophyte who read the article as it stood a month ago would imagine that Freud's work has the same relation to current psychology that Darwin's work has to current biology. \n\nLet's compare them: Darwinism thoroughly permeates modern biology, and his fundamentals, built on copious observation and collegial collaboration, now much extended with new data and new theories, continue to provide accurate predictions and fruitful avenues of exploration. Freud's fundamental ideas were founded on minimal and dubious observations, and his theories do not meet the test of being logical, minimal, reproducible explanations of the data. As far as Jung goes, I haven't gotten to him yet ), and I don't know anything about Maslow. Freud draws my fire precisely because he is so high profile, so influential, and so wrong. I was required to study The Interpretation of Dreams in high school as if it were scientific fact, and the disgust I felt when I later realized it was all as stupid as it sounded has turned me into - yes - a Freud critic. Of course, I am striving for NPOV. Do you really feel that presenting Freud's pseudoscience without comment is NPOV, whereas quoting one of many eminent critics is not? 21:55, 14 Sep 2003 (UTC)"""
4,2084821,It is not just you. This is a laundry list of stupid allegations scooped up from god-knows-where. Probably two-thirds of it has little basis in fact.


In [38]:
tqdm.pandas()
df_test['text'] = df_test['text'].progress_apply(text_cleaning)


  0%|          | 0/7537 [00:00<?, ?it/s]

In [39]:
df_test = clean(df_test,"text")


In [40]:
pd.set_option("display.max_rows",100)
# df_test.head(100)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator

### Training Pipeline

In [42]:

def training_pipeline(data_name, n_folds =CONFIG["nfolds"]):
    if data_name== "ubias":
        n_folds=9
    val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
    val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
    test_preds_arr = np.zeros((df_test.shape[0], n_folds))
#     print(n_folds)
    
    for fld in range(0,n_folds):
        print("\n\n")
        print(f' ****************************** FOLD: {fld} ******************************')
        df = pd.read_csv(f'/kaggle/working/df_{data_name}_fld{fld}.csv')
        print(df.shape)

        vectorizer =  TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))
        pipeline = Pipeline(
            [
                ("vectorizer", vectorizer),

                ("clf", Ridge())
            ]
        )
        print("\nTrain:")
        # Train the pipeline
        pipeline.fit(df['text'].values.astype('U'), df['score'])

        # What are the important features for toxicity

        print('\nTotal number of features:', len(pipeline['vectorizer'].get_feature_names()) )

        feature_wts = sorted(list(zip(pipeline['vectorizer'].get_feature_names(), 
                                      np.round(pipeline['clf'].coef_,2) )), 
                             key = lambda x:x[1], 
                             reverse=True)

        print(feature_wts[:30])

        print("\npredict validation data ")
        val_preds_arr1[:,fld] = pipeline.predict(df_val['less_toxic'])
        val_preds_arr2[:,fld] = pipeline.predict(df_val['more_toxic'])

        print("\npredict test data ")
        test_preds_arr[:,fld] = pipeline.predict(df_test['text'])
        
    return val_preds_arr1,val_preds_arr2, test_preds_arr

In [43]:
class_val_preds_arr1,class_val_preds_arr2, class_test_preds_arr = training_pipeline("classification")





 ****************************** FOLD: 0 ******************************
(25960, 2)

Train:

Total number of features: 106571
[('gay', 2.37), ('kill ', 2.13), (' gay', 2.0), ('nig', 1.95), ('jew', 1.91), (' nl', 1.89), ('l33', 1.87), (' f ', 1.85), (' jew', 1.83), ('ck ', 1.79), (' nig', 1.76), ('nigg', 1.73), (' die', 1.71), (' homo', 1.7), ('die', 1.62), (' g ', 1.6), ('gga', 1.6), ('die ', 1.53), ('ie ', 1.52), ('fuk', 1.5), (' fag', 1.47), (' nigg', 1.41), ('homo', 1.37), ('jew ', 1.37), ('igg', 1.36), (' ck', 1.32), (' ga', 1.32), (' die ', 1.3), (' je', 1.3), (' rap', 1.28)]

predict validation data 

predict test data 



 ****************************** FOLD: 1 ******************************
(25960, 2)

Train:

Total number of features: 107217
[('gay', 2.18), (' g ', 2.15), (' jew', 2.01), ('jew', 2.0), (' f ', 1.88), ('nigg', 1.85), (' gay', 1.84), ('nig', 1.82), ('ck ', 1.77), ('kill ', 1.77), (' die', 1.64), (' nig', 1.62), (' nigg', 1.61), (' homo', 1.59), ('die', 1.51), ('

In [44]:
ruddit_val_preds_arr1,ruddit_val_preds_arr2, ruddit_test_preds_arr = training_pipeline("ruddit")




 ****************************** FOLD: 0 ******************************
(4578, 2)

Train:

Total number of features: 32557
[(' fuc', 0.94), ('fuc', 0.94), ('ck ', 0.93), (' fuck', 0.92), ('fuck', 0.92), ('fuck ', 0.92), ('uck', 0.87), ('uck ', 0.84), (' fu', 0.75), (' ass ', 0.66), (' shit', 0.59), ('shit', 0.59), ('shit ', 0.59), ('hit', 0.54), (' ass', 0.53), ('sex', 0.51), (' shi', 0.5), (' sex', 0.48), ('hit ', 0.47), (' sh', 0.45), ('shi', 0.45), (' dick', 0.42), ('ass', 0.42), ('dick', 0.42), ('dick ', 0.42), (' dic', 0.4), ('ick ', 0.38), ('kill', 0.38), (' as', 0.37), (' du', 0.36)]

predict validation data 

predict test data 



 ****************************** FOLD: 1 ******************************
(4578, 2)

Train:

Total number of features: 32441
[(' fuc', 2.81), ('fuc', 2.81), (' fuck', 2.78), ('fuck', 2.78), ('fuck ', 2.78), ('uck', 2.67), ('ck ', 2.63), ('uck ', 2.6), (' fu', 2.24), (' shit', 1.89), ('shit', 1.89), ('shit ', 1.89), ('hit', 1.75), (' shi', 1.62), ('hit 

In [45]:
ubias_val_preds_arr1,ubias_val_preds_arr2, ubiast_test_preds_arr = training_pipeline("ubias")




 ****************************** FOLD: 0 ******************************
(216044, 2)

Train:

Total number of features: 239294
[(' crap', 5.34), (' ass ', 5.23), (' ass.', 4.57), (' fool', 4.47), ('fool', 4.25), ('darn', 4.19), (' sh ', 4.13), (' jerk', 4.06), ('jerk', 3.89), ('balls', 3.87), ('moron', 3.83), (' moro', 3.77), ('crap', 3.68), ('diot', 3.64), (' suck', 3.51), ('dumb', 3.47), ('idiot', 3.45), ('silly', 3.45), (' darn', 3.39), ('suck', 3.39), ('scum', 3.35), ('damn', 3.32), (' dumb', 3.26), ('fools', 3.2), ('loser', 3.17), (' scum', 3.16), ('orant', 3.16), ('moro', 3.15), ('oser', 3.15), (' liar', 3.14)]

predict validation data 

predict test data 



 ****************************** FOLD: 1 ******************************
(216044, 2)

Train:

Total number of features: 238885
[(' crap', 4.91), (' ass.', 4.69), (' ass ', 4.51), ('fool', 4.37), (' fool', 4.23), ('balls', 4.08), ('moron', 3.93), ('fools', 3.83), ('idiot', 3.67), ('diot', 3.62), (' moro', 3.6), ('crap', 3.59),

### Pipeline Validation

In [46]:
print(" Toxic data ")
p1 = class_val_preds_arr1.mean(axis=1)
p2 = class_val_preds_arr2.mean(axis=1)

print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')

print(" Ruddit data ")
p3 = ruddit_val_preds_arr1.mean(axis=1)
p4 = ruddit_val_preds_arr2.mean(axis=1)

print(f'Validation Accuracy is { np.round((p3 < p4).mean() * 100,2)}')

print(" Unintended Bias data ")
p5 = ubias_val_preds_arr1.mean(axis=1)
p6 = ubias_val_preds_arr2.mean(axis=1)

print(f'Validation Accuracy is { np.round((p5 < p6).mean() * 100,2)}')

 Toxic data 
Validation Accuracy is 67.06
 Ruddit data 
Validation Accuracy is 63.1
 Unintended Bias data 
Validation Accuracy is 68.11


In [47]:
print("Find right weight")

wts_acc = []
for i in range(30,70,1):
    for j in range(0,20,1):
        w1 = i/100
        w2 = (100 - i - j)/100
        w3 = (1 - w1 - w2 )
        p1_wt = w1*p1 + w2*p3 + w3*p5
        p2_wt = w1*p2 + w2*p4 + w3*p6
        wts_acc.append( (w1,w2,w3, 
                         np.round((p1_wt < p2_wt).mean() * 100,2))
                      )
sorted(wts_acc, key=lambda x:x[3], reverse=True)[:5]

Find right weight


[(0.47, 0.34, 0.19, 69.12),
 (0.48, 0.34, 0.18, 69.09),
 (0.46, 0.35, 0.19000000000000006, 69.08),
 (0.48, 0.33, 0.19, 69.08),
 (0.49, 0.32, 0.19, 69.08)]

In [48]:
w1,w2,w3,_ = sorted(wts_acc, key=lambda x:x[2], reverse=True)[0]
#print(best_wts)

p1_wt = w1*p1 + w2*p3 + w3*p5
p2_wt = w1*p2 + w2*p4 + w3*p6

In [49]:
# Predict using pipeline

df_test['score'] = w1*class_test_preds_arr.mean(axis=1) + w2*ruddit_test_preds_arr.mean(axis=1) + w3*ubiast_test_preds_arr.mean(axis=1)


In [50]:
len(df_test), len(df_test["score"].unique())

(7537, 7521)

In [51]:
df_test[df_test["score"].duplicated(keep=False)]

comment_id  \
1832    95080362   
2842   160935265   
3203   181929554   
3204   181930070   
3542   199072816   
3543   199072973   
4757   270227263   
4759   270227815   
4832   275797183   
4833   275812977   
5140   298854514   
5190   301925517   
5743   339264823   
5744   339265390   
5752   339478276   
5753   339478966   
5832   345043812   
5833   345043888   
5852   346641598   
5853   346641762   
6193   375083006   
6194   375157867   
6221   377694722   
6222   377694727   
6947   444588772   
6948   444589429   
7128   463947914   
7130   463947986   
7312   481969878   
7313   481970432   
7503   501720037   
7504   501720124   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [52]:
sub = df_test[["comment_id","score"]]
sub.to_csv("submission.csv", index=False)